# transfer_learning_dogs_vs_cats

El script tiene como objetivo principal demostrar cómo aplicar transfer learning y fine-tuning usando un modelo preentrenado (Xception, entrenado en ImageNet) para realizar la clasificación binaria de imágenes (perros vs gatos), utilizando TensorFlow/Keras.

En detalle, los objetivos del script son:

1. Preparar el dataset:

  * Descargar el conjunto de datos cats_vs_dogs desde tensorflow_datasets.

  * Dividirlo en entrenamiento (40%), validación (10%) y test (10%).

  * Redimensionar todas las imágenes a 150x150 píxeles y normalizarlas a un rango [-1,1].

  * Crear pipelines eficientes con cache(), batch() y prefetch().

2. Aumentar los datos (data augmentation):

  * Aplicar transformaciones aleatorias (flip horizontal y rotación) para incrementar artificialmente el tamaño y variabilidad del dataset.

3. Construir el modelo:

  * Cargar Xception preentrenado en ImageNet como extractor de características (con trainable=False).

  * Agregar un bloque de clasificación con GlobalAveragePooling2D, Dropout y una capa densa final para salida binaria.

  * Integrar directamente la normalización y el aumento de datos como parte del modelo.

4. Entrenar el modelo (fase de transferencia):

  * Entrenar únicamente las capas del nuevo clasificador manteniendo congelado el modelo base.

5. Realizar fine-tuning:

  * Descongelar el modelo base (trainable=True) y entrenar con una tasa de aprendizaje muy baja para ajustar finamente los pesos.

6. Evaluar y predecir:

  * Realizar predicciones sobre batches de imágenes del conjunto de test.

  * Mostrar cómo hacer predicciones sobre imágenes individuales o datos externos (imágenes sueltas cargadas manualmente).

### En resumen:

El script enseña el flujo completo para aplicar transferencia de aprendizaje con un modelo preentrenado en Keras, adaptarlo a un problema nuevo (clasificación de perros y gatos), mejorar su desempeño con fine-tuning y usarlo para predicciones en distintos formatos de entrada.

# Transfer learning y fine-tuning

En este ejemplo utilizaremos modelos preentrenado en Keras para hacer transferencia de aprendizaje desde ImageNet a un set de datos de clasificación de perros y gatos.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

### Obtención de los datos

Para mantener el set de datos pequeño y no caer en sobreentrenamiento, usaremos el 40% de este para entrenamiento (25.000 imágenes), 10% para validaciónn, y 10% para test (no consideren los errores en la descarga, no son un problema).

In [ ]:
import tensorflow_datasets as tfds

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # para incluir las etiquetas
)

print(f"Ejemplos entrenamiento: {tf.data.experimental.cardinality(train_ds)}")
print(f"Ejemplos validación: {tf.data.experimental.cardinality(validation_ds)}")
print(f"Ejemplos test: {tf.data.experimental.cardinality(test_ds)}")

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")

### Normalización de los datos

Las imágenes de este conjunto de datos tienen distintos tamaños. Para corregir esto y normalizar los datos, llevaremos cada imagen a una resolución de 150x150 pixeles, y transformaremos el valor de color (R, G y B) de cada pixel del intervalo [0,255] al [-1,1]. Para hacer esto último, utilizaremos una capa de tipo `Normalization` en la red, que aplica una transformación fija a cada dato de entrada.

In [ ]:
# la capa de normalización ejecuta outputs = (inputs - mean) / sqrt(var)
mean = np.array([127.5] * 3)
var = mean ** 2
norm_layer = keras.layers.Normalization(mean=mean, variance=var)

A continuación, hacemos el _resizing_:

In [ ]:
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Como extra opcional, pero muy práctico, agregamos un mecanismo de `caching` para acelerar la carga de datos.

In [ ]:
batch_size = 32

train_ds_batched = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds_batched = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds_batched = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

### Aumento de datos

Dado que el conjunto de datos es pequeño para el tamaño de una red, utilizaremos un esquema de aumento de datos para incrementar artificialmente la cantidad de esto. En este caso en particular, aplicaremos a cada dato de manera aleatoria, al momento de ser ingresado a la red, un flip horizontal y una rotación. Al igual que antes, esta transformación la modelaremos como capas, de forma de embeberla directamente en la estructura.

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)


Visualicemos algunas transformaciones:

In [ ]:
for images, labels in train_ds_batched.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.axis("off")

## Construcción del modelo

Ahora cargaremos un modelo desde Keras (Xception), que fue entrenado originalmente en ImageNet. Para hacer la transferencia, incluimos el aumento de datos, la normalización y finalmente un grupo de capas para hacer la clasificación en el nuevo set de datos (pooling, dropout, capa densa).

Para que todo esto funcione en modo de transferencia, es fundamental setear el modelo importado con `trainable=False`, de forma que sea utilizado como un extractor de características y que lo único que se entrene sean las capas del nuevo clasificador.

In [ ]:
base_model = keras.applications.Xception(weights="imagenet",
                                         input_shape=(150, 150, 3),
                                         #no incluimos el clasificador
                                         include_top=False)

base_model.trainable = False
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)
x = norm_layer(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

## Entrenamiento del modelo (solo capas de clasificación)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(train_ds_batched, epochs=epochs, validation_data=validation_ds_batched)

## Fine-tuning

Finalmente, haremos un par de _epochs_ de fine-tuning, cuidando setear el modelo ahora en `trainable=True`. Otro aspecto relevante es el learning rate, que es mantenido en un valor bajo para evitar el sobrenetrenamiento.

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # learning rate bajo
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 5
model.fit(train_ds_batched, epochs=epochs, validation_data=validation_ds_batched)

Si bien la mejor en rendimiento no es increíble, esta sí es medible. Es importante notar como se reduce rapidamente el valor de la pérdida, lo que indica un alto riesgo de sobreentrenamiento si se continua con el proceso por más _epochs_.

## Predicción
Existen múltiples formas de hacer predicción utilizando un modelo ya entrenado. Algo que siempre es fundamental es asegurarse que los datos de entrada estén en el formato adecuado. En este caso, que las imágenes tenga la dimensión correcta.

### Predicción en base a batches
Si queremos hacer predicción sobre los ejemplos de test del set de datos con que entrenamos, basta con pedirle a `test_ds_batched` un batch (32 elementos) y luego aplicar la función `predict`.

In [ ]:
batch = test_ds_batched.take(1)
prediction = model.predict(batch) > 0

In [ ]:
for images in batch:
  plt.figure(figsize=(20, 20))
  for i in range(batch_size):
      ax = plt.subplot(4, 8, i + 1)
      plt.imshow(images[0][i].numpy().astype("int32"))
      plt.title(int(prediction[i]))
      plt.axis("off")

### Predicción sobre ejemplos que no están en un batch
Muchas veces, los ejemplos no se encontrarán organizados en batches, ya sea porque no han sido procesados para que tengan ese formato, a pesar de pertenecer al mismo conjunto de datos que el usado para entrenar, o porque vienen de otra fuente. Cualquiera sea el caso, a pesar de que el modelo esté compilado para batches de un tamaño, igualmente podrá predecir sobre un conjunto de menor o mayor tamaño.

In [ ]:
images = test_ds.take(9)
prediction = model.predict(images) > 0

In [ ]:
import tensorflow as tf
import numpy as np

images_list = []
labels_list = []

for batch_images, batch_labels in test_ds:
    # Si el batch es de una sola imagen, expandir dimensión:
    if len(batch_images.shape) == 3:
        images_list.append(batch_images.numpy())
        labels_list.append(batch_labels.numpy())
    else:
        images_list.extend(batch_images.numpy())
        labels_list.extend(batch_labels.numpy())

    if len(images_list) >= 9:
        break

# Tomar exactamente 9 imágenes y etiquetas
images = np.stack(images_list[:9])
labels = np.array(labels_list[:9])

# Realizar la predicción
predictions = model.predict(images) > 0


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i, img in enumerate(images):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img.astype("int32"))
    plt.title(int(predictions[i]))  # Asegúrate de que predictions está alineado con images
    plt.axis("off")


Esto funciona incluso para ejemplos sueltos subidos a colab.

In [ ]:
img = tf.keras.utils.load_img('supuestamente_un_perro.jpg')
img = tf.keras.utils.img_to_array(img)
plt.figure(figsize=(10, 10))
ax = plt.imshow(img.astype("int32"))

In [ ]:
img = tf.image.resize(img, size)
plt.figure(figsize=(10, 10))
ax = plt.imshow(img.numpy().astype("int32"))

In [ ]:
prediction = model.predict(tf.expand_dims(img, 0)) > 0

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img.numpy().astype("int32"))
ax = plt.title(int(prediction))